In [1]:
file0 = """################################
#########.######################
#########..#####################
#########..G####################
########....#GG#################
########G......#################
########........################
###.####...#....################
#....###.###...G.###############
##......####.....#.#G..#.#######
###G.G...###.........#...#######
###......##...........##########
#............G#####...##########
#..G##G......#######..##########
#.G.#.......#########..#########
####..G.....#########...#.######
#...........#########..........#
##.#.....#..#########.E.....E..#
##.###..G.G.#########..........#
##...........#######E.#.......##
#.............#####..........###
#....#.....E................####
##.............##.E...........##
#....G.G.................###..##
#..............#.....E...###..##
#..##.##.G.....##E.......###.###
###G..##.......###.###...##...##
#####.E##.E.G..######...E.#..###
####...###..#..#######.......###
####...###############.#########
#####..#########################
################################"""

In [2]:
file1 = """#########
#G..G..G#
#.......#
#.......#
#G..E..G#
#.......#
#.......#
#G..G..G#
#########"""

r = [list(i) for i in file1.split('\n')]
r

[['#', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', 'G', '.', '.', 'G', '.', '.', 'G', '#'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
 ['#', 'G', '.', '.', 'E', '.', '.', 'G', '#'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
 ['#', 'G', '.', '.', 'G', '.', '.', 'G', '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#']]

In [67]:
class Node():
    def __init__(self,x,y, parent = None):
        self.x = x
        self.y = y
        self.parent = parent
        self.childs = []
        self.leaf = False
        self.length = 0
        
        
        
    def __repr__(self):
        return f'({self.x},{self.y})'
    
    def add_child(self,child):
        self.childs.append(child)
        
    def get_range(self,rd):
        rango = []
        
        if rd[self.y+1][self.x] == '.':
            rango.append((self.x,self.y+1))
            
        if rd[self.y-1][self.x] == '.':
            rango.append((self.x,self.y-1))
            
        if rd[self.y][self.x-1] == '.':
            rango.append((self.x-1,self.y))
            
        if rd[self.y][self.x+1] == '.':
            rango.append((self.x+1,self.y))
            
        return rango
    
    def make_length(self):
        if self.parent is None:
            return self.length
        self.length+=1
        return self.parent.make_length()
    
    def get_length(self):
        self.length = 0
        self.make_length()
        return self.length
    
    def get_x(self):
        if self.parent.parent.parent is None:
            return self.parent.x
        return self.parent.get_x()
    
    def get_y(self):
        if self.parent.parent.parent is None:
            return self.parent.y
        return self.parent.get_y()
    
    
class Path():
    def __init__(self,master_node,rd, goals):
        self.master_node = master_node
        self.rd = rd
        self.current_level = 0
        self.nodes = {self.current_level:[master_node]}
        self.leafs = []
        self.posiciones= [(master_node.x, master_node.y)]
        self.goals = goals
        self.fin = False
        
    def add_node(self,node):
        self.nodes.append(node)
        
    def next_level(self):
        to_add = []
        
        self.nodes[self.current_level+1] = []
        for node in self.nodes[self.current_level]:
            for p in node.get_range(self.rd):
                if p not in self.posiciones:
                    
                    child = Node(p[0],p[1],node)
                    node.add_child(child)
                    self.nodes[self.current_level+1].append(child)
                    to_add.append(p)
                    if p in self.goals:
                        child.leaf = True
                        self.leafs.append(child)
        if len(self.nodes[self.current_level+1])==0:
            self.fin = True
        if len(to_add) == 0:
            self.fin = True
        self.posiciones += to_add
        self.current_level += 1
        
    def fill(self):
        while (len(self.leafs) == 0) and (self.fin == False ):
            self.next_level()
        return self.next_move()
    
    def next_move(self):
        if self.fin == True:
            return None

        ne =  sorted([c for 
                       c in self.leafs],
              key = lambda n: (n.get_length(),
                               n.get_y(),n.get_x()))[0]
        return ne.get_x(),ne.get_y()
            

class Unit():
    def __init__(self,  x, y, tipo):
        self.attack_power = 3
        self.hp = 200
        self.tipo = tipo 
        self.x = x
        self.y = y
        self.alive = True
        
    def attack(self,other_unit):
        other_unit.hp-=self.attack_power
        if other_unit.hp <=0:
            other_unit.alive = False
            
    def get_range(self,rd):
        rango = []
        if rd[self.y][self.x+1] == '.':
            rango.append((self.x+1, self.y))
        if rd[self.y][self.x-1] == '.':
            rango.append((self.x-1, self.y))
        if rd[self.y+1][self.x] == '.':
            rango.append((self.x, self.y+1))
        if rd[self.y-1][self.x] == '.':
            rango.append((self.x, self.y-1))
            
        return rango
    
    
    def __repr__(self):
        return f'[{self.tipo},{self.hp}]({self.x},{self.y})'
    

        

In [164]:
class Game():
    def __init__(self,rd):
        
        self.rd = [list(i) for i in rd.split('\n')]
        
    def build_units(self):
        self.units = []
        for n,fila in enumerate(self.rd):
            for m,columna in enumerate(fila):
                if columna == 'G':
                    self.units.append(Unit(m,n,'G'))
                if columna == 'E':
                    self.units.append(Unit(m,n,'E'))
        self.units.sort(key = lambda u: (u.y,u.x))
        
    def get_enemies(self,unit):
        return sorted([u for u in self.units 
                       if u.alive and u.tipo != unit.tipo],
                     key = lambda p: (p.hp,p.y,p.x))
    
    def get_enemies_in_range(self,unit):
        enemies_in_range = []
        for enemy in self.get_enemies(unit):
            if (enemy.x,enemy.y) == (unit.x+1,unit.y):
                enemies_in_range.append(enemy)
                
            if (enemy.x,enemy.y) == (unit.x-1,unit.y):
                enemies_in_range.append(enemy)
                
            if (enemy.x,enemy.y) == (unit.x,unit.y+1):
                enemies_in_range.append(enemy)
                
            if (enemy.x,enemy.y) == (unit.x,unit.y-1):
                enemies_in_range.append(enemy)
        
        return sorted( enemies_in_range,
                     key = lambda p: (p.hp,p.y,p.x))
    
    def get_goals(self,unit):
        goals = []
        for u in self.units:
            if u.tipo != unit.tipo:
                goals += u.get_range(self.rd)
        return goals
    
                
                
                    
    
    def pass_turn(self):
        for u in self.units:
            print('current_unit:' ,u)
            if not u.alive:
                continue
            #possible scenarios:
            # a) has a unit to attack -> attack
            # has no goals: continue
            # has no available movements: continue
            # can move: move
            
            #a) has a unit to attack:
            if len(self.get_enemies_in_range(u))>0:
                enemy = self.get_enemies_in_range(u)[0]
                print(u,' attacks ',enemy)
                u.attack(enemy)
                if not enemy.alive:
                    self.rd[enemy.y][enemy.x] = '.'
                continue
            
            if len(self.get_goals(u)) == 0:
                continue
                
                
            if len(u.get_range(self.rd))==0:
                continue
            
            #if goals are in range, no need to fill path:
            
            if len([
                p for p in u.get_range(self.rd) if p in self.get_goals(u)
            ])>0:
                #sort goals that are in range
                new_x,new_y = sorted([g for g in self.get_goals(u) 
                        if g in u.get_range(self.rd)],
                       key = lambda m: (m[1],m[0])
                      )[0]
                print(u,'->')
                self.rd[u.y][u.x] = '.'
                u.x,u.y = new_x,new_y
                self.rd[u.y][u.x] = u.tipo
                continue
            
            if len(u.get_range(self.rd))>0:
                pati = Path(Node(u.x,u.y),self.rd,self.get_goals(u))

                try:
                    new_x,new_y = pati.fill()
                except:
                    print(u,'failed try')
                    continue

                print(u,'->')

                self.rd[u.y][u.x] = '.'
                u.x,u.y = new_x,new_y
                self.rd[u.y][u.x] = u.tipo
                
            if len(self.get_enemies_in_range(u))>0:
                enemy = self.get_enemies_in_range(u)[0]
                print(u,' attacks ',enemy)
                u.attack(enemy)
                if not enemy.alive:
                    self.rd[enemy.y][enemy.x] = '.'
                continue
            
            
        
    
    def play(self):
        turno = 0
        self.build_units()
        while (len([u for u in self.units if u.tipo == 'G'])>0) and\
        (len([u for u in self.units if u.tipo == 'E'])>0):
            
            self.pass_turn()
            turno+=1
            print(turno,end = ', ')
            self.units = sorted([u for u in self.units if u.alive],
                            key = lambda p: (p.y,p.x))
            for i in mg.rd:
                print(''.join(i))
        
            

In [165]:
mg = Game(file1)
mg.build_units()
mg.units
mg.play()

current_unit: [G,200](1,1)
[G,200](1,1) ->
current_unit: [G,200](4,1)
[G,200](4,1) ->
current_unit: [G,200](7,1)
[G,200](7,1) ->
current_unit: [G,200](1,4)
[G,200](1,4) ->
current_unit: [E,200](4,4)
[E,200](4,4) ->
current_unit: [G,200](7,4)
[G,200](7,4) ->
current_unit: [G,200](1,7)
[G,200](1,7) ->
current_unit: [G,200](4,7)
[G,200](4,7) ->
current_unit: [G,200](7,7)
[G,200](7,7) ->
1, #########
#.G...G.#
#...G...#
#...E..G#
#.G.....#
#.......#
#G..G..G#
#.......#
#########
current_unit: [G,200](2,1)
[G,200](2,1) ->
current_unit: [G,200](6,1)
[G,200](6,1) ->
current_unit: [G,200](4,2)
[G,200](4,2)  attacks  [E,200](4,3)
current_unit: [E,197](4,3)
[E,197](4,3)  attacks  [G,200](4,2)
current_unit: [G,200](7,3)
[G,200](7,3) ->
current_unit: [G,200](2,4)
[G,200](2,4) ->
current_unit: [G,200](1,6)
[G,200](1,6) ->
current_unit: [G,200](4,6)
[G,200](4,6) ->
current_unit: [G,200](7,6)
[G,200](7,6) ->
2, #########
#..G.G..#
#...G...#
#.G.E.G.#
#.......#
#G..G..G#
#.......#
#.......#
#########


In [154]:
file_exp = """#######
#G..#E#
#E#E.E#
#G.##.#
#...#E#
#...E.#
#######"""

In [155]:
mg = Game(file_exp)
mg.build_units()
mg.units
mg.play()

current_unit: [G,200](1,1)
[G,200](1,1)  attacks  [E,200](1,2)
current_unit: [E,200](5,1)
current_unit: [E,197](1,2)
[E,197](1,2)  attacks  [G,200](1,1)
current_unit: [E,200](3,2)
[E,200](3,2) ->
current_unit: [E,200](5,2)
[E,200](5,2) failed try
current_unit: [G,200](1,3)
[G,200](1,3)  attacks  [E,197](1,2)
current_unit: [E,200](5,4)
[E,200](5,4) failed try
current_unit: [E,200](4,5)
[E,200](4,5) ->
1, #######
#G.E#E#
#E#..E#
#G.##.#
#...#E#
#..E..#
#######
current_unit: [G,197](1,1)
[G,197](1,1)  attacks  [E,194](1,2)
current_unit: [E,200](3,1)
[E,200](3,1) ->
current_unit: [E,200](5,1)
current_unit: [E,191](1,2)
[E,191](1,2)  attacks  [G,197](1,1)
current_unit: [E,200](5,2)
[E,200](5,2) failed try
current_unit: [G,200](1,3)
[G,200](1,3)  attacks  [E,191](1,2)
current_unit: [E,200](5,4)
[E,200](5,4) failed try
current_unit: [E,200](3,5)
[E,200](3,5) ->
2, #######
#GE.#E#
#E#..E#
#G.##.#
#..E#E#
#.....#
#######
current_unit: [G,194](1,1)
[G,194](1,1)  attacks  [E,188](1,2)
current_uni

In [134]:
for i in mg.rd:
    print(''.join(i))

#######
#...#E#
#E#...#
#.E##.#
#E..#E#
#.....#
#######


In [135]:
mg.units

[[E,200](5,1), [E,194](1,2), [E,182](2,3), [E,200](1,4), [E,200](5,4)]

In [74]:
pati.nodes

{0: [(5,2)], 1: [(5,3), (4,2)], 2: []}

In [12]:
600+194+182

976

In [166]:
file_exp2 = """#######
#.G...#
#...EG#
#.#.#G#
#..G#E#
#.....#
#######"""

In [167]:
mg = Game(file_exp2)
mg.build_units()
mg.units

[[G,200](2,1),
 [E,200](4,2),
 [G,200](5,2),
 [G,200](5,3),
 [G,200](3,4),
 [E,200](5,4)]

In [168]:
for i in mg.rd:
    print(''.join(i))

#######
#.G...#
#...EG#
#.#.#G#
#..G#E#
#.....#
#######


In [169]:
mg.pass_turn()

current_unit: [G,200](2,1)
[G,200](2,1) ->
current_unit: [E,200](4,2)
[E,200](4,2)  attacks  [G,200](5,2)
current_unit: [G,197](5,2)
[G,197](5,2)  attacks  [E,200](4,2)
current_unit: [G,200](5,3)
[G,200](5,3)  attacks  [E,200](5,4)
current_unit: [G,200](3,4)
[G,200](3,4) ->
current_unit: [E,197](5,4)
[E,197](5,4)  attacks  [G,200](5,3)


In [125]:
mg.units

[[G,200](3,1),
 [E,197](4,2),
 [G,197](5,2),
 [G,200](3,3),
 [G,197](5,3),
 [E,197](5,4)]

In [126]:
for i in mg.rd:
    print(''.join(i))

#######
#..G..#
#...EG#
#.#G#G#
#...#E#
#.....#
#######


In [127]:
mg.pass_turn()

current_unit: [G,200](3,1)
[G,200](3,1) ->
current_unit: [E,197](4,2)
[E,197](4,2)  attacks  [G,197](5,2)
current_unit: [G,194](5,2)
[G,194](5,2)  attacks  [E,197](4,2)
current_unit: [G,200](3,3)
[G,200](3,3) ->
current_unit: [G,197](5,3)
[G,197](5,3)  attacks  [E,197](5,4)
current_unit: [E,194](5,4)
[E,194](5,4)  attacks  [G,197](5,3)


In [128]:
for i in mg.rd:
    print(''.join(i))

#######
#...G.#
#..GEG#
#.#.#G#
#...#E#
#.....#
#######


In [129]:
mg.units

[[G,200](4,1),
 [G,200](3,2),
 [E,194](4,2),
 [G,194](5,2),
 [G,194](5,3),
 [E,194](5,4)]

In [99]:
mg.pass_turn()

current_unit: [G,200](4,1)
current_unit: [E,191](4,2)
current_unit: [G,191](5,2)
current_unit: [G,194](5,3)
current_unit: [G,200](3,2)
current_unit: [E,191](5,4)
